In [2]:
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.utils import to_categorical
from DESSA import DESSA
from tensorflow.keras.preprocessing.image import ImageDataGenerator

sys.path.append('../Metaheuristic Algorithms/')  # Ensure the DESSA module is accessible


In [3]:
# Create an instance of the ImageDataGenerator for data normalization and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,   # Normalize pixel values to be between 0 and 1
    validation_split=0.2  # Use 20% of the data for validation
)

# Define the path to your dataset directory
train_directory = '/Users/issackondreddy/Desktop/Education/Projects/Cancer Detection/Data/Dataset'

# Setup the training data generator
train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(32, 32),  # Resize images to 32x32 for model compatibility
    batch_size=4,  # Smaller batch size for quick testing
    class_mode='categorical',  # Multi-class classification
    subset='training',  # Use the training subset of images
    shuffle=True  # Shuffle the images to reduce model bias
)

# Setup the validation data generator
validation_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=(32, 32),
    batch_size=4,  # Consistent batch size with training for simplicity
    class_mode='categorical',
    subset='validation',  # Use the validation subset of images
    shuffle=True
)

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [4]:
def create_cnn_model(params, input_shape=(224, 224, 3), num_classes=4):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)  # We add a fully-connected layer as before
    predictions = Dense(num_classes, activation='softmax')(x)
    
    # Freeze all the layers in the base VGG16 model
    for layer in base_model.layers:
        layer.trainable = False
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
bounds = [(16, 64), (2, 5), (0.1, 0.5)]  # Example bounds, adjust as needed

dessa_optimizer = DESSA(lambda params: -create_cnn_model(params).evaluate(validation_generator, steps=1)[1],
                        bounds, population_size=5, iterations=1)
best_params, best_fitness, fitness_history = dessa_optimizer.optimize()
print("Best Parameters:", best_params)
print("Best Validation Accuracy:", -best_fitness)

1/1 [==============================] - 0s 220ms/step - loss: 1.8265 - accuracy: 0.2500
Best Parameters: [64.          3.0952419   0.25979188]
Best Validation Accuracy: 1.0


In [6]:
final_model = create_cnn_model([64, 3.1, 0.26], input_shape=(224, 224, 3), num_classes=4)
final_history = final_model.fit(train_generator, epochs=20, validation_data=validation_generator, verbose=1)
print("Final model accuracy on validation set:", final_history.history['val_accuracy'][-1])

Epoch 1/20
1281/1281 [==============================] - 59s 46ms/step - loss: 0.9496 - accuracy: 0.5368 - val_loss: 1.2699 - val_accuracy: 0.3104
Epoch 2/20
1281/1281 [==============================] - 68s 53ms/step - loss: 0.8956 - accuracy: 0.5657 - val_loss: 1.1773 - val_accuracy: 0.4034
Epoch 3/20
1281/1281 [==============================] - 62s 49ms/step - loss: 0.8727 - accuracy: 0.5741 - val_loss: 1.2301 - val_accuracy: 0.4285
Epoch 4/20
1281/1281 [==============================] - 68s 53ms/step - loss: 0.8547 - accuracy: 0.5899 - val_loss: 1.2796 - val_accuracy: 0.3792
Epoch 5/20
1281/1281 [==============================] - 76s 60ms/step - loss: 0.8474 - accuracy: 0.5886 - val_loss: 1.2126 - val_accuracy: 0.4058
Epoch 6/20
1281/1281 [==============================] - 55s 43ms/step - loss: 0.8296 - accuracy: 0.5973 - val_loss: 1.1541 - val_accuracy: 0.4636
Epoch 7/20
1281/1281 [==============================] - 55s 43ms/step - loss: 0.8196 - accuracy: 0.6028 - val_loss: 1.2513 -